In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import seaborn as sns
cmap = sns.color_palette()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

we can see here that weights column in assets_details will be used to get the weighted pearson correlation coefficient evaluation metric.

In [ ]:
assets = pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv")
assets.sort_values("Weight", ascending=False)

Whole Distribution of weights of different assets into pie chart

In [ ]:
assets["weight_percent"] = (assets["Weight"] / assets["Weight"].sum()) * 100
assets.sort_values("Weight", ascending=False)

Get The Pie chart for above weight_percentage feature

In [ ]:
fig = px.pie(assets, values='weight_percent', names='Asset_Name', title='Weights assigned to each cryptocurrency')
fig.show()

In [ ]:
assets.columns

In [ ]:
asset_names_dict = {row["Asset_Name"]:row["Asset_ID"] for ind, row in assets.iterrows()}

In [ ]:
asset_names_dict

In [ ]:
train = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
train.head()

In [ ]:
asset_names = [
    'Bitcoin',
    'Ethereum',
    'Cardano',
    'Binance Coin',
    'Dogecoin',
    'Bitcoin Cash',
    'Litecoin',
    'Ethereum Classic',
    'Stellar',
    'TRON',
    'Monero',
    'EOS.IO',
    'IOTA',
    'Maker'
]

time_frame = []
for asset in asset_names:
    #fetching the asset whose id is similar to any particular asset in that dataframe and gather into one dataframe
#     print(coin_df)
    coin_df = train[train["Asset_ID"]==asset_names_dict[asset]].set_index("timestamp")
    start_time = coin_df.index[0].astype('datetime64[s]')
#     print(start_time)
    end_time = coin_df.index[-1].astype('datetime64[s]')
#     print(end_time)
    time_frame.append([asset, start_time, end_time])
time_df = pd.DataFrame(time_frame)
time_df.columns = ["Asset_Name", "Start_Time", "End_Time"]
time_df

Inference from above Observation:

* The earliest start date in the data is Jan 1, 2018 and most coins have that start datetime.
* The data is available till Sep 21, 2021 and all the coins have the same end datetime.
* Dogecoin has the least historical information available of the given coins in the given data and it is captured only from April 2019 instead of others assets.

Missing data


In [ ]:
missing_list = []
for asset in asset_names:
    coin_df =train[train["Asset_ID"]==asset_names_dict[asset]].set_index("timestamp")
    missing_list.append([asset, coin_df.shape[0]] + coin_df.isna().sum().tolist())
missing_df = pd.DataFrame(missing_list)
missing_df.columns = ["Asset_Name", "TotalRows", 'Missing_Asset_ID', 'Missing_Count', 'Missing_Open', 
                      'Missing_High', 'Missing_Low', 'Missing_Close', 'Missing_Volume', 'Missing_VWAP', 'Missing_Target']
missing_df

Inference from above missing data Information:

* There is a lot of missing data in the Target column
* All the other columns do not have missing data
* Bitcoin has the least number of missing target values and Monero the most.


In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv',usecols=['Target', 'Asset_ID','timestamp'], dtype={'Asset_ID': 'int8'})
train_df['datetime'] = pd.to_datetime(train_df['timestamp'], unit='s')
train_df = train_df.set_index('datetime').drop('timestamp', axis=1)
train_df = train_df[(train_df.index.year == 2021) & (train_df.index.month > 5)]
dfs = {asset_id: train_df[train_df['Asset_ID'] == asset_id].resample('1min').interpolate().copy() for asset_id in train_df['Asset_ID'].unique()}
del train_df
for df_test, df_pred in iter_test:
    df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
    for _, row in df_test.iterrows():
        try:
            df = dfs[row['Asset_ID']]
            closest_train_sample = df.iloc[df.index.get_loc(row['datetime'], method='nearest')]
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = closest_train_sample['Target']
        except:
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
    df_pred['Target'] = df_pred['Target'].fillna(0)
    env.predict(df_pred)